<img src="images/banner3.png" width="100%" />

<font face="Calibri">
<br>
<font size="5"> <b>Sand clustering with Silhouette Analysis and KMeans notebook</b></font>

<br>
<font size="4"> <b> Nicolas Pucino; PhD Student @ Deakin University, Australia </b> <br>
<img style="padding:7px;" src="images/sandpiper_sand_retouched.png" width="170" align="right" /></font>

<font size="3">This notebook illustrates how to use Sandpiper to perform Silhouette Analysis and KMeans on all previously extracted points. <br>

<b>This notebook covers the following concepts:</b>

- Silhouete Analysis.
- KMeans clustering.
</font>


</font>

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np

from sandpyper.outils import coords_to_points 
from sandpyper.labels import get_sil_location, get_opt_k, kmeans_sa

Loading the project-related lists

- loc codes
- crs dict string

In [2]:
# The location codes used troughout the analysis
loc_codes=["mar","leo"]

# The Coordinate Reference Systems used troughout this study
crs_dict_string= {
                 'mar': {'init': 'epsg:32754'},
                 'leo': {'init': 'epsg:32755'},
                 }

### Loading, merging and preparing the tables

The function __get_merged_table__ merge the rgb and z tables together and format it in a way it is digestible for further analysis.

In [16]:
%%time

#Loading the tables

rgb_table_path=r"C:\my_packages\doc_data\profiles\rgb.csv"
z_table_path=r"C:\my_packages\doc_data\profiles\elevation.csv"

rgb_table=gpd.read_file(rgb_table_path)
z_table=gpd.read_file(z_table_path)

# As the distance (across-transect) comes from an interpolation, it has too many digits.
# let's round both tables distance columns to 2 significant values and assign their data type as "float".

rgb_table["distance"]=np.round(rgb_table.loc[:,"distance"].values.astype("float"),2)
z_table["distance"]=np.round(z_table.loc[:,"distance"].values.astype("float"),2)

C:\conda3\envs\sandpiper_env\lib\site-packages\geopandas\geodataframe.py:422: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for feature in features_lst:


Wall time: 16.9 s


Storing Geodataframes as CSV is handy, but __we lose the column data type information__.
Especially important is the __geometry column__, which we need to convert back into __Shapely Point object format__.
To do that, the function __coords_to_points__ can be used across a Series ('geometry'). It can take quite a bit of time, so, if you have a lot of points, get ready!

In [17]:
rgb_table['geometry']=rgb_table.coordinates.apply(coords_to_points)
z_table['geometry']=z_table.coordinates.apply(coords_to_points)

In [18]:
# Here, we merge the two tables (storing elevation and rgb information)

data_merged = pd.merge(z_table,rgb_table[["band1","band2","band3","point_id"]],on="point_id",validate="one_to_one")

# replace empty values with np.NaN
data_merged=data_merged.replace("", np.NaN)

# and convert the z column into floats.
data_merged['z']=data_merged.z.astype("float")

In [19]:
# Here, we add two features, slope and curvature, computed from the elevation series,
# in case we wnat to use for KMeans clustering.
# Note that when passing from one transect to another, slope and curvature computations are wrong.
# However, we will clip those areas as they are in the water or in the backdune.

data_merged["slope"]=np.gradient(data_merged.z)
data_merged["curve"]=np.gradient(data_merged.slope)

In [20]:
# Our rasters have NaN values set to -32767.0. Thus, we replace them with np.Nan.
data_merged.z.replace(-32767.0,np.nan,inplace=True)


The __get_sil_location__ function will iteratively perform KMeans clustering and Silhouette Analysis with increasing number of clusters (k, specified in the `ks` parameter) for every survey, using the feature set specified in the parameter `feature_set`.

This will return a dataframe with Average Silhouette scores with different k for all surveys, which we use to find sub-optimal number of clusters with __get_opt_k__ function.

Then, with the sub-optimal k, we finally run KMeans with __kmeans_sa__ function on all the surveys to obtain clustered points to visually discriminate between sand and non-sand in a Qgis environment.

In [21]:
%%time
# Run interatively KMeans + SA

feature_set=["band1","band2","band3","slope","curve"]
sil_df=get_sil_location(data_merged,
                        ks=(2,30), 
                        feature_set=feature_set,
                       random_state=10)

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

Working on : mar, 2019-05-16.


  0%|          | 0/28 [00:00<?, ?it/s]

C:\conda3\envs\sandpiper_env\lib\site-packages\sklearn\cluster\_kmeans.py:793: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 1.0 (renaming of 0.25).
  " removed in 1.0 (renaming of 0.25).", FutureWarning)


...

For n_clusters = 27 The average silhouette_score is : 0.3007054556094827


C:\conda3\envs\sandpiper_env\lib\site-packages\sklearn\cluster\_kmeans.py:793: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 1.0 (renaming of 0.25).
  " removed in 1.0 (renaming of 0.25).", FutureWarning)


For n_clusters = 28 The average silhouette_score is : 0.29219020455362954


C:\conda3\envs\sandpiper_env\lib\site-packages\sklearn\cluster\_kmeans.py:793: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 1.0 (renaming of 0.25).
  " removed in 1.0 (renaming of 0.25).", FutureWarning)


For n_clusters = 29 The average silhouette_score is : 0.2970891262785888
Wall time: 4min 45s


In [22]:
# Find sub-optimal k by searching inflexion points where
# an additional cluster do not considerably degrade the overall clustering performance.

opt_k=get_opt_k(sil_df, sigma=0 )
opt_k

{'leo_2018-06-06': 10,
 'leo_2018-07-13': 9,
 'leo_2018-09-20': 11,
 'leo_2019-02-11': 10,
 'leo_2019-03-28': 10,
 'leo_2019-07-31': 10,
 'leo_2019-09-03': 11,
 'leo_2019-10-14': 9,
 'leo_2019-12-12': 9,
 'leo_2020-02-21': 10,
 'leo_2020-05-11': 8,
 'leo_2020-06-24': 9,
 'leo_2020-08-10': 10,
 'leo_2020-09-21': 10,
 'leo_2020-11-06': 5,
 'mar_2018-06-01': 5,
 'mar_2018-06-21': 5,
 'mar_2018-07-27': 5,
 'mar_2018-09-25': 5,
 'mar_2018-11-13': 5,
 'mar_2018-12-11': 5,
 'mar_2019-02-05': 5,
 'mar_2019-03-13': 5,
 'mar_2019-05-16': 4}

If we are not satisfied with the sub-optimal k returned by the algorithm, we can manually specify each survey k
by defining a dictionary.

In [9]:
# Based on our observations on a dataset comprising 87 surveys, 10 clusters (k=10) is generally a good tradeoff.

opt_k={'leo_2018-06-06': 10,
 'leo_2018-07-13': 10,
 'leo_2018-09-20': 10,
 'leo_2019-02-11': 10,
 'leo_2019-03-28': 10,
 'leo_2019-07-31': 10,
 'mar_2018-06-01': 10,
 'mar_2018-06-21': 10,
 'mar_2018-07-27': 10,
 'mar_2018-09-25': 10,
 'mar_2018-11-13': 10,
 'mar_2018-12-11': 10,
 'mar_2019-02-05': 10,
 'mar_2019-03-13': 10,
 'mar_2019-05-16': 10}

In [23]:
# Clustering the dataset with optimal k

data_classified=kmeans_sa(data_merged,opt_k, feature_set=feature_set)

C:\conda3\envs\sandpiper_env\lib\site-packages\sandpyper\labels\labels.py:280: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_merged.dropna(inplace=True)


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

C:\conda3\envs\sandpiper_env\lib\site-packages\sklearn\cluster\_kmeans.py:793: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 1.0 (renaming of 0.25).
  " removed in 1.0 (renaming of 0.25).", FutureWarning)
C:\conda3\envs\sandpiper_env\lib\site-packages\sandpyper\labels\labels.py:324: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_in["label_k"]=clusterer.fit_predict(minmax_scaled_df)
C:\conda3\envs\sandpiper_env\lib\site-packages\sklearn\cluster\_kmeans.py:793: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 1.0 (renaming of 0.25).
  " removed in 1.0 (renaming of 0.25).", FutureWarning)
C:\conda3\envs\sandpiper_env\lib\site-packages\sandpyper\labels\labels.py:324: SettingWithCopyWarn

  0%|          | 0/15 [00:00<?, ?it/s]

C:\conda3\envs\sandpiper_env\lib\site-packages\sklearn\cluster\_kmeans.py:793: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 1.0 (renaming of 0.25).
  " removed in 1.0 (renaming of 0.25).", FutureWarning)
C:\conda3\envs\sandpiper_env\lib\site-packages\sandpyper\labels\labels.py:324: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_in["label_k"]=clusterer.fit_predict(minmax_scaled_df)
C:\conda3\envs\sandpiper_env\lib\site-packages\sklearn\cluster\_kmeans.py:793: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 1.0 (renaming of 0.25).
  " removed in 1.0 (renaming of 0.25).", FutureWarning)
C:\conda3\envs\sandpiper_env\lib\site-packages\sandpyper\labels\labels.py:312: SettingWithCopyWarn

C:\conda3\envs\sandpiper_env\lib\site-packages\sandpyper\labels\labels.py:312: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_in["label_k"]=clusterer.fit_predict(minmax_scaled_df)
C:\conda3\envs\sandpiper_env\lib\site-packages\sklearn\cluster\_kmeans.py:793: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 1.0 (renaming of 0.25).
  " removed in 1.0 (renaming of 0.25).", FutureWarning)
C:\conda3\envs\sandpiper_env\lib\site-packages\sandpyper\labels\labels.py:312: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html

In [24]:
data_classified=pd.merge(data_classified[["point_id","label_k"]],data_merged, how="left", on="point_id", validate="one_to_one")
data_classified

,point_id,label_k,distance,z,tr_id,raw_date,coordinates,location,survey_date,x,y,geometry,band1,band2,band3,slope,curve
0,67148080l2690700eo10,2,1.0,1.096543,47,20180606,POINT (299874.2117248897 5773731.971115951),leo,2018-06-06,299874.2117248897,5773731.971115951,POINT (299874.212 5773731.971),123.0,126.0,121.0,-0.010084,-0.010396
1,67141080l2630750eo10,6,1.5,1.076090,47,20180606,POINT (299874.7086046137 5773732.026888166),leo,2018-06-06,299874.7086046137,5773732.026888166,POINT (299874.709 5773732.027),92.0,86.0,97.0,-0.033487,-0.006814
2,67143080l2670800eo20,6,2.0,1.029569,47,20180606,POINT (299875.2054843378 5773732.08266038),leo,2018-06-06,299875.20548433776,5773732.08266038,POINT (299875.205 5773732.083),90.0,84.0,94.0,-0.023713,0.016813
3,67146080l2610850eo20,8,2.5,1.028664,47,20180606,POINT (299875.7023640618 5773732.138432594),leo,2018-06-06,299875.7023640618,5773732.138432594,POINT (299875.702 5773732.138),98.0,94.0,101.0,0.000139,0.017553
4,67148080l2650800eo30,8,3.0,1.029846,47,20180606,POINT (299876.1992437858 5773732.194204807),leo,2018-06-06,299876.1992437858,5773732.194204807,POINT (299876.199 5773732.194),109.0,106.0,118.0,0.011393,-0.001586
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56091,60101091m2575700ar25,2,25.5,1.972202,0,20190516,POINT (731471.4863936177 5705142.796794743),mar,2019-05-16,731471.4863936177,5705142.796794743,POINT (731471.486 5705142.797),211.0,205.0,183.0,-0.040461,-0.003882
56092,60106091m2506900ar20,2,26.0,1.942527,0,20190516,POINT (731471.9834328609 5705142.742462518),mar,2019-05-16,731471.9834328609,5705142.742462518,POINT (731471.983 5705142.742),210.0,206.0,182.0,-0.045471,-0.002531
56093,60100091m2546200ar25,2,26.5,1.881259,0,20190516,POINT (731472.4804721042 5705142.688130293),mar,2019-05-16,731472.4804721042,5705142.688130293,POINT (731472.480 5705142.688),201.0,195.0,169.0,-0.045523,0.003576
56094,60104091m2577400ar20,2,27.0,1.851482,0,20190516,POINT (731472.9775113474 5705142.633798068),mar,2019-05-16,731472.9775113474,5705142.633798068,POINT (731472.978 5705142.634),209.0,202.0,176.0,-0.038319,0.001305


### GOOD!

save the __data_classified__ dataframe as a CSV file and head to the __SANDPYPER polygon correction notebook__.

In [25]:
data_classified.to_csv(r"C:\my_packages\doc_data\labels\data_classified.csv", index=False)

___